In [3]:
import os
os.chdir("/home/ec2-user/scVI/")
os.getcwd()

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from umap import UMAP

use_cuda = True
from scvi.dataset.scanorama import DatasetSCANORAMA
from scvi.dataset.dataset import GeneExpressionDataset
import numpy as np
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
import matplotlib.pyplot as plt
import seaborn as sns
from scvi.metrics.clustering import select_indices_evenly
from sklearn.manifold import TSNE
dirs = (
        open('/data/scanorama/conf/panorama.txt')
        .read().rstrip().split()
)

In [3]:
datasets = [DatasetSCANORAMA(d) for d in dirs]

all_dataset = GeneExpressionDataset.concat_datasets(*datasets)
# Keeping 5216 genes

labels = (open('/data/scanorama/data/cell_labels/all.txt').read().rstrip().split())
all_dataset.cell_types,all_dataset.labels = np.unique(labels,return_inverse=True)
all_dataset.labels = all_dataset.labels.reshape(len(all_dataset.labels),1)
all_dataset.batch_indices = all_dataset.batch_indices.astype('int')

from scvi.harmonization.utils_chenling import trainVAE,VAEstats
# full = trainVAE(all_dataset, 'scanorama', 1, nlayers=3,n_hidden=256)
full = trainVAE(all_dataset, 'scanorama', 0) #  nlayers=2,n_hidden=128
# full = trainVAE(all_dataset, 'scanorama', 2, nlayers=3,n_hidden=128)


Preprocessing dataset: /data/scanorama/data/293t_jurkat/293t
Preprocessing dataset: /data/scanorama/data/293t_jurkat/jurkat
Preprocessing dataset: /data/scanorama/data/293t_jurkat/jurkat_293t_50_50
Preprocessing dataset: /data/scanorama/data/293t_jurkat/jurkat_293t_99_1
Preprocessing dataset: /data/scanorama/data/brain/neuron_9k
Preprocessing dataset: /data/scanorama/data/macrophage/infected
Preprocessing dataset: /data/scanorama/data/macrophage/mixed_infected
Preprocessing dataset: /data/scanorama/data/macrophage/uninfected
Preprocessing dataset: /data/scanorama/data/macrophage/uninfected_donor2
Preprocessing dataset: /data/scanorama/data/hsc/hsc_mars
Preprocessing dataset: /data/scanorama/data/hsc/hsc_ss2
Preprocessing dataset: /data/scanorama/data/pancreas/pancreas_inDrop
Preprocessing dataset: /data/scanorama/data/pancreas/pancreas_multi_celseq2_expression_matrix
Preprocessing dataset: /data/scanorama/data/pancreas/pancreas_multi_celseq_expression_matrix
Preprocessing dataset: /dat

In [5]:
# for 250 iterations, takes 45:14 to train VAE
latent, batch_indices, labels, stats = VAEstats(full)


LL : 661761409032014844247796613120.0000


In [6]:
celltype_names = ['293t', 'CD19+ B cells', 'Hsc CMP', 'Hsc GMP', 'PBMC CD14+ monocyte', 'Hsc MEP', 'PBMC CD56+ NK', 'PBMC T cells', 'PBMC CD4 T cells', 'PBMC CD8 T cells','Hsc Unknown', 'Pancreas Acinar', 'Pancreas Alpha', 'Pancreas Beta', 'Pancreas Beta(ER stress)', 'Pancreas Delta', 'Pancreas Ductal','Pancreas Endothelial', 'Pancreas Gamma', 'Jurkat', 'Macrophage', 'Neuron', 'Pancreas Unknown','Pancreas  Stellate']


In [8]:
dict(zip(all_dataset.cell_types, celltype_names))

{'293t': '293t',
 'B': 'CD19+ B cells',
 'CMP': 'Hsc CMP',
 'GMP': 'Hsc GMP',
 'M': 'PBMC CD14+ monocyte',
 'MEP': 'Hsc MEP',
 'NK': 'PBMC CD56+ NK',
 'T': 'PBMC T cells',
 'TCD4': 'PBMC CD4 T cells',
 'TCD8': 'PBMC CD8 T cells',
 'U': 'Hsc Unknown',
 'acinar': 'Pancreas Acinar',
 'alpha': 'Pancreas Alpha',
 'beta': 'Pancreas Beta',
 'beta_er': 'Pancreas Beta(ER stress)',
 'delta': 'Pancreas Delta',
 'ductal': 'Pancreas Ductal',
 'endo': 'Pancreas Endothelial',
 'gamma': 'Pancreas Gamma',
 'jurkat': 'Jurkat',
 'macrophage': 'Macrophage',
 'neuron': 'Neuron',
 'other': 'Pancreas Unknown',
 'stellate': 'Pancreas  Stellate'}

In [9]:
# 1:05:58 for the more complex model
    # , clustering_scores,clustering_accuracy
plotname='scanorama'
sample = select_indices_evenly(2000, batch_indices)
colors = sns.color_palette('bright') + \
         sns.color_palette('muted') + \
         sns.color_palette('pastel') + \
         sns.color_palette('dark') + \
         sns.color_palette('colorblind')

from umap import UMAP

latent_s = latent[sample, :]
label_s = labels[sample]
batch_s = batch_indices[sample]
if latent_s.shape[1] != 2:
    latent_s = UMAP().fit_transform(latent_s)

keys = all_dataset.cell_types
fig, ax = plt.subplots(figsize=(20, 18))
key_order = np.argsort(keys)
for i, k in enumerate(key_order):
    ax.scatter(latent_s[label_s == k, 0], latent_s[label_s == k, 1], c=colors[i % 30], label=celltype_names[k],
               edgecolors='none')
    ax.legend(bbox_to_anchor=(1.1, 0.5), borderaxespad=0, fontsize='x-large')
fig.patch.set_visible(False)
ax.axis('off')
fig.tight_layout()
plt.savefig('../' + plotname + '.labelText.pdf')



In [10]:
dataset_names = ['293t', 'Jurkat', 'Jurkat 293t 50:50', 'Jurkat 293t 99:1',
 'Neuron9k',
 'Macrophage Infected', 'Macrophage Mixed', 'Macrophage Uninfected', 'Macrophage Uninfected2',
 'HSC Mars', 'HSC SS2',
 'Pancreas inDrop', 'Pancreas Celseq2', 'Pancreas Celseq', 'Pancreas Fluidigm C1', 'Pancreas SS2',
 'PBMC 68K', 'B cells', 'CD14 Monocytes', 'CD4 T helper', 'CD56 NK', 'Cytotoxic T cells', 'Memory T cells', 
 'Regulatory T cells', 'PBMC Kang', 'PBMC 10X']


In [11]:


keys = dirs
fig, ax = plt.subplots(figsize=(20, 14))
key_order = np.argsort(keys)

colors = sns.light_palette("navy",4, reverse=True) + \
sns.light_palette("gray",1, reverse=True) + \
sns.light_palette("orange",4, reverse=True) + \
sns.light_palette("purple",2,reverse=True) + \
sns.light_palette("green",5,reverse=True) + \
sns.light_palette("red",10,reverse=True)

for k in key_order:
    ax.scatter(latent_s[batch_s == k, 0], latent_s[batch_s == k, 1], c=colors[k], label=dataset_names[k],
               edgecolors='none')
    ax.legend(bbox_to_anchor=(1.1, 0.5), borderaxespad=0, fontsize='x-large')
fig.patch.set_visible(False)
ax.axis('off')
fig.tight_layout()
plt.savefig('../' + plotname + '.batchesText.pdf')


In [ ]:
from scvi.metrics.clustering import clustering_scores
res_knn = clustering_scores(np.asarray(latent), labels, 'knn')
res_kmeans = clustering_scores(np.asarray(latent), labels, 'KMeans')

plt.figure(figsize=(5, 4))
order = np.argsort(res_knn['clusteracc'])
plt.bar(np.arange(len(all_dataset.cell_types)), np.asarray(res_knn['clusteracc'])[order])
plt.xticks(np.arange(len(all_dataset.cell_types)), np.asarray(celltype_names)[order],rotation='vertical')
plt.tight_layout()
plt.savefig("/data/scanorama/acc_barplot.pdf")
